In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [2]:
code = """
#include <stdio.h>
#include <stdlib.h>
#define N 1000000
__global__ void add(int *a, int *b, int *c) {
int tid = blockIdx.x * blockDim.x + threadIdx.x;
if (tid < N) {
c[tid] = a[tid] + b[tid];
}
}
int main() {
int *a, *b, *c;
int *d_a, *d_b, *d_c;
int size = N * sizeof(int);
// Allocate memory on host
a = (int*)malloc(size);
b = (int*)malloc(size);
c = (int*)malloc(size);
// Initialize arrays
for (int i = 0; i < N; i++) {
a[i] = i;
b[i] = i * 2;
}
// Allocate memory on device
cudaMalloc(&d_a, size);
cudaMalloc(&d_b, size);
cudaMalloc(&d_c, size);
// Copy data from host to device
cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);
// Launch kernel with 1 million threads
add<<<(N + 255) / 256, 256>>>(d_a, d_b, d_c);
// Copy result from device to host
cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);
// Print first and last elements of result
printf("c[0]=%d, c[%d] = %d",c[0],N-1,c[N-1]);
// Free memory
free(a);
free(b);
free(c);
cudaFree(d_a);
cudaFree(d_b);
cudaFree(d_c);
return 0;
}
"""



In [3]:
text_file = open("assign4.cu","w")
text_file.write(code)
text_file.close()

In [4]:
!nvcc assign4.cu

In [5]:
!./a.out

c[0]=0, c[999999] = 2999997

In [6]:
!nvprof ./a.out

==2542== NVPROF is profiling process 2542, command: ./a.out
==2542== Profiling application: ./a.out
c[0]=0, c[999999] = 2999997==2542== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   53.02%  1.7608ms         1  1.7608ms  1.7608ms  1.7608ms  [CUDA memcpy DtoH]
                   45.56%  1.5131ms         2  756.54us  723.21us  789.87us  [CUDA memcpy HtoD]
                    1.41%  46.943us         1  46.943us  46.943us  46.943us  add(int*, int*, int*)
      API calls:   97.21%  242.46ms         3  80.821ms  101.42us  242.22ms  cudaMalloc
                    2.09%  5.2132ms         3  1.7377ms  982.08us  3.2379ms  cudaMemcpy
                    0.38%  939.56us         1  939.56us  939.56us  939.56us  cuDeviceGetPCIBusId
                    0.23%  585.16us         3  195.05us  153.87us  219.07us  cudaFree
                    0.06%  151.39us       101  1.4980us     180ns  61.488us  cuDeviceGetAttribute
                